In [1]:
import pandas as pd

In [2]:
book = pd.read_csv('DictionaryData/book.csv', sep=">")
word = pd.read_csv('DictionaryData/word.csv', sep=">")
relation_book_word = pd.read_csv('DictionaryData/relation_book_word.csv', sep=">")
print(len(book), list(book.columns))
print(len(word), list(word.columns))
print(len(relation_book_word), list(relation_book_word.columns))


583 ['bk_id', 'bk_parent_id', 'bk_level', 'bk_order', 'bk_name', 'bk_item_num', 'bk_direct_item_num', 'bk_author', 'bk_book', 'bk_comment', 'bk_orgnization', 'bk_publisher', 'bk_version', 'bk_flag']
70164 ['vc_id', 'vc_vocabulary', 'vc_phonetic_uk', 'vc_phonetic_us', 'vc_frequency', 'vc_difficulty', 'vc_acknowledge_rate']
1044580 ['bv_id', 'bv_book_id', 'bv_voc_id', 'bv_flag', 'bv_tag', 'bv_order']


In [3]:
from database import operation as op
op.BookSeries(**{
  'bk_level': 1, 'bk_order': 6.0, 'bk_name': '大学英语四级词汇', 
  'bk_item_num': 74062, 'bk_direct_item_num': 0, 'bk_author': '',
  'bk_book': '', 'bk_comment': '', 'bk_organization': '',
  'bk_publisher': '', 'bk_version': '', 'bk_flag': '',
  'bk_series_id': 'c9f0f895fb98ab9159f51fd0'
})

BookSeries(bk_level=1, bk_order=6.0, bk_name='大学英语四级词汇', bk_item_num=74062, bk_direct_item_num=0, bk_author='', bk_book='', bk_comment='', bk_organization='', bk_publisher='', bk_version='', bk_flag='', bk_series_id='c9f0f895fb98ab9159f51fd0')

In [10]:
from typing import List

from fastapi import Depends, FastAPI, HTTPException
from sqlalchemy.orm import Session

from database import SessionLocal, engine, Base

Base.metadata.create_all(bind=engine)

app = FastAPI()

# Dependency
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


In [11]:
# init database
from database import model, schema, operation as op

s = SessionLocal()
book.fillna(value='', inplace=True)
word.fillna(value='', inplace=True)
relation_book_word.fillna(value='', inplace=True)
for i in book.to_dict(orient='records'):
    i['bk_organization'] = i['bk_orgnization']
    del i['bk_orgnization']
    if i['bk_parent_id'] == '0':
        del i['bk_parent_id']
        i['bk_series_id'] = i['bk_id']
        del i['bk_id']
        s.add(schema.BookSeries(**i))
    else:
        del i['bk_parent_id']
        s.add(schema.Book(**i))
s.commit()
for i in word.to_dict(orient='records'):
    s.add(schema.Word(**i))
s.commit()
for i in relation_book_word.to_dict(orient='records'):
    s.add(schema.Unit(**i))
s.commit()

In [ ]:
# rel_set = set(relation_book_word['bv_book_id'])
# for i in list(relation_book_word['bv_book_id']):
#     if i in rel_set:
#         rel_set.remove(i)
#     else:
#         print(i)

In [6]:
relation_book_word.to_dict(orient='records')[:2]

[{'bv_id': '58450c828958a37d5c10f763',
  'bv_book_id': 'd645920e395fedad7bbbed0e',
  'bv_voc_id': '57067b9ca172044907c615d7',
  'bv_flag': 4,
  'bv_tag': 'Unit 1',
  'bv_order': 1},
 {'bv_id': '58450c828958a37d5c10f764',
  'bv_book_id': 'd645920e395fedad7bbbed0e',
  'bv_voc_id': '57067c3ca172044907c64fe2',
  'bv_flag': 1,
  'bv_tag': 'Unit 1',
  'bv_order': 2}]

In [7]:
book.to_dict(orient='records')[:2]

[{'bk_id': 'c9f0f895fb98ab9159f51fd0',
  'bk_parent_id': '0',
  'bk_level': 1,
  'bk_order': 6.0,
  'bk_name': '大学英语四级词汇',
  'bk_item_num': 74062,
  'bk_direct_item_num': 0,
  'bk_author': '',
  'bk_book': '',
  'bk_comment': '',
  'bk_orgnization': '',
  'bk_publisher': '',
  'bk_version': '',
  'bk_flag': ''},
 {'bk_id': '45c48cce2e2d7fbdea1afc51',
  'bk_parent_id': '0',
  'bk_level': 1,
  'bk_order': 0.0,
  'bk_name': '小学英语',
  'bk_item_num': 15233,
  'bk_direct_item_num': 0,
  'bk_author': '',
  'bk_book': '',
  'bk_comment': '',
  'bk_orgnization': '',
  'bk_publisher': '',
  'bk_version': '',
  'bk_flag': ''}]

In [ ]:
# book.to_sql(name='raw_book', con=engine)
# word.to_sql(name='raw_word', con=engine)
# relation_book_word.to_sql(name='raw_relation_book_word', con=engine)

In [ ]:
# book.to_excel('DictionaryData/book.xlsx', index=False)
# word.to_excel('DictionaryData/word.xlsx', index=False)
# relation_book_word.to_excel('DictionaryData/relation_book_word.xlsx', index=False)

In [ ]:
word.columns

Index(['vc_id', 'vc_vocabulary', 'vc_phonetic_uk', 'vc_phonetic_us',
       'vc_frequency', 'vc_difficulty', 'vc_acknowledge_rate'],
      dtype='object')

In [ ]:
batch_size = 5
book_name = "雅思词汇词以类记"
book_id = book[book['bk_name'] == book_name]['bk_id'].values[0]
word_batch = []
relations_in_book = relation_book_word[relation_book_word['bv_book_id'] == book_id]
for i in range(0, len(relations_in_book), batch_size):
    relation = relations_in_book[i:i+batch_size]
    batch = relation.join(word.set_index('vc_id'), on='bv_voc_id')
    # batch = word[word['vc_id'] == relation['bv_voc_id'].values[0]]
    batch = batch[['vc_vocabulary', 'vc_phonetic_uk', 'vc_phonetic_us', 
                   'vc_frequency', 'vc_difficulty', 'vc_acknowledge_rate']]
    word_batch.append(batch)
len(word_batch)

886

In [ ]:
word_batch[0]

,vc_vocabulary,vc_phonetic_uk,vc_phonetic_us,vc_frequency,vc_difficulty,vc_acknowledge_rate
386204,arid,[ˈærɪd],[ˈærɪd],0.301901,9,0.286239
386205,atmosphere,[ˈætməsfɪə],[ˈætməsfɪɚ],0.619064,3,0.746754
386206,balmy,[ˈbɑ:mɪ],[ˈbɑmi],0.330187,7,0.265653
386207,barometer,[bəˈrɒmɪtə],[bəˈrɑmɪtɚ],0.345525,6,0.233177
386208,blast,[blɑ:st],[blæst],0.667218,7,0.359326


In [ ]:
from dataclasses import dataclass
from collections import defaultdict, deque
from typing import Any, List, Mapping, Optional, Set, Dict, Tuple

from langchain import GoogleSearchAPIWrapper
import logging
import os
import json
import re
import time
import cmd
import random
import sys
import traceback
import ast
from loguru import logger
from pydantic import BaseModel, Field, validator
import click
import tiktoken
from langchain.agents import AgentOutputParser
from langchain.output_parsers import (
    PydanticOutputParser,
    OutputFixingParser,
    RetryOutputParser,
)
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, CombinedMemory, ConversationSummaryMemory
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.callbacks.file import FileCallbackHandler
from langchain.llms.base import LLM
from langchain.tools import BaseTool, StructuredTool
from langchain.prompts import PromptTemplate
from langchain.agents.mrkl.prompt import FORMAT_INSTRUCTIONS


from LLM import OpenAIChat
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain.memory.chat_message_histories import StreamlitChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate



log_id = 0

def create_chat_message(role: str, content: str):
    """
    Create a chat message with the given role and content.

    Args:
    role (str): The role of the message sender, e.g., "system", "user", or "assistant".
    content (str): The content of the message.

    Returns:
    dict: A dictionary containing the role and content of the message.
    """
    return {"role": role, "content": content}

llm = OpenAIChat(model_name="gpt-3.5-turbo" , temperature=0.5)

# os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")
os.environ["SERPER_API_KEY"] = "ab74e952491d09c9007e0fa14c28da535dbec672"

In [ ]:
TEMPLATE = """\
please write a story at least 5 sentences long, using the words [{words}].
The word should be in the same order as the words in the prompt.
The word should be surrounded by square brackets.
Please response in the following format.
English: ...[word1]...[word2]...
Chinese: ...[单词1]...[单词2]..
"""
prompt_template = PromptTemplate(
    template=TEMPLATE,
    input_variables=["words"],
    partial_variables={
    }
)
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True, 
)
words = ", ".join(list(word_batch[0]["vc_vocabulary"].values))
ans = chain.run(words)
print(ans)



> Entering new LLMChain chain...
Prompt after formatting:
please write a story at least 5 sentences long, using the words [arid, atmosphere, balmy, barometer, blast].
The word should be in the same order as the words in the prompt.
The word should be surrounded by square brackets.
Please response in the following format.
English: ...[word1]...[word2]...
Chinese: ...[单词1]...[单词2]..


> Finished chain.
English: The [arid] desert landscape was unrelenting in its heat, with the [atmosphere] feeling like a suffocating blanket. But as night fell, the temperature dropped and a [balmy] breeze swept through, providing some relief. Checking the [barometer], it seemed that a storm was brewing, and sure enough, a sudden [blast] of wind and rain hit, drenching everything in its path. Despite the chaos, the desert was grateful for the much-needed moisture. 

Chinese: [干旱的]沙漠景观的热量无情，[气氛]让人感到窒息。但夜幕降临时，温度下降，[温暖的]微风吹过，提供了一些缓解。检查了[气压计]，似乎有一场风暴正在酝酿，果然，一阵突然的[爆炸]风雨袭来，淋湿了一切。尽管混乱，但沙漠仍然感激这场急需的降雨。


In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath('.'))
sys.path.append(os.path.dirname(SCRIPT_DIR))


In [2]:
import gradio as gr
from database.operation import *
from memorize import *
from database import SessionLocal, engine, Base

Base.metadata.create_all(bind=engine)
db = SessionLocal()

users = get_all_users(db)
# book_series_list = get_book_series_list(db)
books = get_all_books(db)

print(users, books)


[<User linxy59@qq.com>] [<Book 人教版高中英语1 - 必修>, <Book 人教版高中英语2 - 必修>, <Book 人教版高中英语3 - 必修>, <Book 人教版高中英语4 - 必修>, <Book 人教版高中英语5 - 必修>, <Book 人教版高中英语6 - 选修>, <Book 人教版高中英语8 - 选修>, <Book 人教版高中英语9 - 选修>, <Book 人教版高中英语10 - 选修>, <Book 新概念英语第一册（新版）>, <Book 朗文3000常用交流词汇>, <Book 人教版高中英语11 - 选修>, <Book 新概念英语第二册（新版）>, <Book 新概念英语第三册（新版）>, <Book 新概念英语第四册（新版）>, <Book TOEFL托福45天突破版>, <Book 星火四级词汇周计划>, <Book GRE词汇精选（NEW）>, <Book 六级词汇乱序版>, <Book 专八词汇乱序版>, <Book 雅思词汇加强版>, <Book BEC中级词汇精选>, <Book 词汇进阶 BASIC>, <Book 词汇进阶 6000>, <Book BEC高级词汇精选>, <Book GMAT词汇红宝书>, <Book 词汇进阶 12000>, <Book BEC初级词汇精选>, <Book 雅思词汇便携版>, <Book 星火四级巧记速记乱序版>, <Book 星火全新专8词汇周计划>, <Book 专八词汇正序版>, <Book 王陆807雅思词汇写作第2版>, <Book BEC词汇乱序版>, <Book 专八词汇便携版>, <Book 四级词汇正序版>, <Book 词汇进阶 23000>, <Book 王陆807雅思词汇阅读第2版>, <Book 六级词汇便携版>, <Book 英语四、六级考试词汇必备>, <Book 淘金4级词汇周计划乱序版>, <Book 新概念英语青少版入门级A>, <Book 新概念英语青少版入门级B>, <Book 淘金4级词汇连环记乱序版>, <Book 思思大王六级单词白金版>, <Book 思思大王四级单词白金版>, <Book 全新版大学英语1 (第二版)>, <Book 全新版大学英语4 (第二版)>, <Book 思思大王高考单词一笑而过

In [2]:
# 1. 创建记忆计划
block_create_user_book = gr.Blocks()

with block_create_user_book:
    batch_size = gr.State(value=10)
    
    select_user = gr.Dropdown(
        [f"{user.email}  [{user.id}]" for user in users], label="用户", info=""
    )
    # select_book_series = gr.Dropdown(
    #     book_series_list, label="系列", info="选择一个系列"
    # )
    select_book = gr.Dropdown(
        [f"{book.bk_name}  [{book.bk_id}]" for book in books],
        # books,
        label="单词书", info="选择一本单词书"
    )
    # def on_select_user(user):
    #     select_user.
    #     return user
    # select_user.change(on_select_user, inputs=[], outputs=[])
    batch_size = gr.Number(value=10)
    title = gr.TextArea(value='单词书', lines=1)
    btn = gr.Button("创建记忆计划")
    status = gr.Textbox("", lines=1)
    def submit(user, book, title, batch_size):
        user_id = user.split(" [")[1][:-1]
        book_id = book.split(" [")[1][:-1]
        user_book = create_user_book(db, UserBookCreate(
            owner_id=user_id,
            book_id=book_id,
            title=title,
            batch_size=batch_size
        ))
        if user_book is not None:
            return "success"
        else:
            return "fail"

    btn.click(submit, [select_user, select_book, title, batch_size], [status])

block_create_user_book.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [3]:

# 选择并批量单词
block_select_and_batching_the_words = gr.Blocks()

with block_select_and_batching_the_words:
    select_user = gr.Dropdown(
        [f"{user.email}  [{user.id}]" for user in users], label="用户", info=""
    )
    select_user_book = gr.Dropdown(
        [], label="记忆计划", info="选择用户的记忆计划"
    )
    known_words = gr.CheckboxGroup(
        [], label="已学会的单词", info="正式记忆前将去除已学会的单词，提高每个批次的新词密度，进而提高效率"
    )
    def on_select_user(user):
        print('user', user)
        if user is None:
            return gr.Dropdown.update(choices=[])
        new_options =  []
        user_id = user.split(" [")[1][:-1]
        user_book = get_user_books_by_owner_id(db, user_id)
        new_options = [f"{book.title} | {book.batch_size}个单词一组 [{book.id}]" for book in user_book]
        return gr.Dropdown.update(choices=new_options)

    def on_select_user_book(user_book):
        print('user_book', user_book)
        if user_book is None:
            return gr.CheckboxGroup.update(choices=[])
        new_options = []
        user_book_id = user_book.split(" [")[1][:-1]
        user_book = get_user_book(db, user_book_id)
        book_id = user_book.book_id
        book = get_book(db, book_id)
        if book is None:
            return gr.CheckboxGroup.update(choices=new_options)
        words = get_words_for_book(db, user_book)
        new_options = [f"{word.vc_vocabulary}" for word in words]
        return gr.CheckboxGroup.update(choices=new_options)
    
    select_user.select(on_select_user, inputs=[select_user], outputs=[select_user_book])
    select_user_book.select(on_select_user_book, inputs=[select_user_book], outputs=[known_words])

    btn = gr.Button("生成批次", elem_id="btn", elem_classes=["abc", "def"])
    status = gr.Textbox("", lines=1)

    def submit(user_book, known_words):
        user_book_id = user_book.split(" [")[1][:-1]
        user_book = get_user_book(db, user_book_id)
        all_words = get_words_for_book(db, user_book)
        unknown_words = []
        for w in all_words:
            if w.vc_vocabulary not in known_words:
                unknown_words.append(w)
        track(db, user_book, unknown_words[:10])
        return "success"

    btn.click(submit, [select_user_book, known_words], [status])

block_select_and_batching_the_words.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


user linxy59@qq.com  [443af7e1-5a7d-422f-8bb4-5fea35c44718]
user_book 雅思词汇词以类记 | 5个单词一组 [780582d3-c59b-4cee-86cb-15d7eab4a964]


> Entering new LLMChain chain...
Prompt after formatting:
please write a story at least 5 sentences long, using the words [['arid', 'atmosphere', 'balmy', 'barometer', 'blast']].
The word should be in the same order as the words in the prompt.
The word should be surrounded by square brackets.
Please response the story in the following format.
English: ...[word1]...[word2]...
Chinese: ...[单词1]...[单词2]..

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties"

In [11]:
import pandas as pd
from collections import defaultdict
# 选择并批量单词
block_remember = gr.Blocks()

with block_remember:
    select_user = gr.Dropdown(
        [f"{user.email}  [{user.id}]" for user in users], label="用户", info=""
    )
    select_user_book = gr.Dropdown(
        [], label="记忆计划", info="请选择记忆计划"
    )
    memorizing_dataframe = gr.Dataframe(
        headers=["单词", "意思", "id", "记忆量"],
        datatype=["str", "str", "str", "str"],
        col_count=(4, "fixed"),
        wrap=True,
    )
    batches = gr.State(value=[])
    current_batch_index = gr.State(value=-1)
    with gr.Row():
        # story = gr.HighlightedText([])
        # translated_story = gr.HighlightedText([])
        # story = gr.Textbox()
        # translated_story = gr.Textbox()
        story = gr.Markdown()
        translated_story = gr.Markdown()
    
    memorize_action = gr.CheckboxGroup(choices=[], label="记住的单词", info="能够复述出意思才算记住")
    with gr.Row():
        previous_batch_btn = gr.Button("上一批")
        next_batch_btn = gr.Button("下一批")
    progress = gr.Slider(0, 1, value=0, step=1, label="进度", info="")

    def on_select_user(user):
        print('user', user)
        if user is None:
            return gr.Dropdown.update(choices=[])
        new_options =  []
        user_id = user.split(" [")[1][:-1]
        user_book = get_user_books_by_owner_id(db, user_id)
        new_options = [f"{book.title} | {book.batch_size}个单词一组 [{book.id}]" for book in user_book]
        return gr.Dropdown.update(choices=new_options)

    def process(story: str):
        # return [(i, "") for i in story.split(" ")]
        return story.replace("[", "**").replace("]", "**")
    def update_batch(memorizing_batch: UserMemoryBatch):
        new_options = []
        word_df = []
        # print(get_user_memory_batch(db, memorizing_batch.id))
        # print(memorizing_batch.id)
        # print(get_user_memory_words_by_batch_id(db, memorizing_batch.id))
        # print(get_words_by_ids(db, [w.word_id for w in get_user_memory_words_by_batch_id(db, memorizing_batch.id)]))
        # words = get_words_in_batch(db, memorizing_batch.id)
        # words = get_words_by_ids(db, [w.word_id for w in memorizing_words])
        memorizing_words = get_user_memory_words_by_batch_id(db, memorizing_batch.id)
        word_to_memorizing = {mw.word_id:mw.id for mw in memorizing_words}
        memorizing_to_word = {mw.id:mw.word_id for mw in memorizing_words}
        words = get_words_by_ids(db, [w.word_id for w in memorizing_words])
        actions = get_actions_at_each_word(db, [w.id for w in memorizing_words])
        remember_count = defaultdict(int)
        forget_count = defaultdict(int)
        for a in actions:
            if a.action == "remember":
                remember_count[memorizing_to_word[a.user_memory_word_id]] += 1
            else:
                forget_count[memorizing_to_word[a.user_memory_word_id]] += 1
        for w in words:
            new_options.append(f"{w.vc_vocabulary}")
            word_df.append([w.vc_vocabulary, w.vc_translation, word_to_memorizing[w.vc_id], f"{remember_count[w.vc_id]} / {remember_count[w.vc_id] + forget_count[w.vc_id]}"])
        df = pd.DataFrame(word_df, columns=["单词", "意思", "id", "记忆量"])
        print(df)
        print(new_options)
        story = memorizing_batch.story
        story = process(story)
        print(story)
        translated_story = memorizing_batch.translated_story
        translated_story = process(translated_story)
        print(translated_story)
        # df = gr.DataFrame.update(
        #     value=df,
        #     max_rows=len(df),
        # )
        return (df, story, translated_story, gr.CheckboxGroup.update(choices=new_options))

    def on_select_user_book(user_book: str):
        """
        1. 当前单词
        2. 对当前单词的操作
        3. 故事
        """
        print('user_book', user_book)
        if user_book is None:
            return [], gr.CheckboxGroup.update(choices=[])
        user_book_id: str = user_book.split(" [")[1][:-1]
        user_book = get_user_book(db, user_book_id)
        batches = get_user_memory_batches_by_user_book_id(db, user_book_id)
        batch_id = user_book.memorizing_batch
        memorizing_batch = get_user_memory_batch(db, batch_id)
        current_batch_index = -1
        if memorizing_batch is not None:
            for index, b in enumerate(batches):
                if b.id == memorizing_batch.id:
                    current_batch_index = index
                    break
        if current_batch_index == -1:
            current_batch_index = 0
            memorizing_batch = batches[0]
            batch_id = memorizing_batch.id
            user_book.memorizing_batch = batch_id
            update_user_book(db, user_book_id, UserBookUpdate(
                owner_id=user_book.owner_id,
                book_id=user_book.book_id,
                title=user_book.title,
                batch_size=user_book.batch_size,
                memorizing_batch=batch_id
            ))
        updates = update_batch(memorizing_batch)
        print(len(batches))
        return (batches, current_batch_index) + updates + (
                gr.Slider.update(
                    minimum=1,
                    maximum=len(batches),
                    value=current_batch_index,
                ),)
    batch_widget = [memorizing_dataframe, story, translated_story, memorize_action]
    select_user.select(on_select_user, inputs=[select_user], outputs=[select_user_book])
    select_user_book.select(
        on_select_user_book,
        inputs=[select_user_book],
        outputs=[batches, current_batch_index] + batch_widget + [progress]
    )
    def submit_batch(batches, current_batch_index):
        memorizing_batch = batches[current_batch_index]
        updates = update_batch(memorizing_batch)
        return updates + (gr.Slider.update(value=current_batch_index+1),)
    def previous_batch(batches, current_batch_index):
        if current_batch_index <= 0:
            current_batch_index = 0
        elif current_batch_index > 0:
            current_batch_index -= 1
        return submit_batch(batches, current_batch_index)
    def next_batch(batches: List[UserMemoryBatch], current_batch_index: int, memorizing_dataframe: pd.DataFrame, memorize_action: List[str]):
        old_index = current_batch_index
        if current_batch_index >= len(batches)-1:
            current_batch_index = len(batches)-1
        elif current_batch_index < len(batches) - 1:
            current_batch_index += 1
        if current_batch_index != old_index:
            # 下一页之前需要保存记忆进度 
            print("下一页之前需要保存记忆进度")
            print(memorizing_dataframe)
            print(memorize_action)
            old_batch = batches[old_index]
            actions = []
            for i in range(len(memorizing_dataframe)):
                row = memorizing_dataframe.iloc[i]
                word_i = row["单词"]
                memorizing_i = row["id"]
                if word_i in memorize_action:
                    actions.append((memorizing_i, "remember"))
                else:
                    actions.append((memorizing_i, "forget"))
            memory_batch_action(db, actions)
        return submit_batch(batches, current_batch_index)
    previous_batch_btn.click(
        previous_batch, 
        inputs=[batches, current_batch_index],
        outputs=batch_widget + [progress]
    )
    next_batch_btn.click(
        next_batch,
        inputs=[batches, current_batch_index, memorizing_dataframe, memorize_action],
        outputs=batch_widget + [progress]
    )

block_remember.launch()


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


user linxy59@qq.com  [443af7e1-5a7d-422f-8bb4-5fea35c44718]
user_book 雅思词汇词以类记 | 5个单词一组 [780582d3-c59b-4cee-86cb-15d7eab4a964]
           单词                                     意思   
0       blast  n.一阵(风),一股(气流),爆炸,冲击波vt.爆炸,,毁灭,使枯萎,损害  \
1  atmosphere                             n.大气,空气,气氛   
2   barometer                                  n.气压计   
3       balmy                adj.芳香的,温和的,止痛的,(空气)温和的   
4        arid               adj.干旱的,贫瘠的(土地等),无趣的,沉闷的   

                                     id    记忆量  
0  409d4f1a-6f1f-49cc-99ea-899afac09391  0 / 2  
1  fe659943-bb5e-4dea-9154-0b7336c7dc05  1 / 2  
2  8b968f99-923c-4863-b18b-ac14824fb0cc  0 / 2  
3  f8923df6-779e-49b7-9fa2-1caa7a8fcceb  0 / 2  
4  bc26f24d-fc42-4ee6-9cc2-959dd549ec34  0 / 2  
['blast', 'atmosphere', 'barometer', 'balmy', 'arid']
The **arid** desert **atmosphere** was suddenly interrupted by a **balmy** **blast** of wind. The **barometer** dropped as sand and debris flew through the air. The travelers huddled toget

In [20]:
import gradio as gr
import os
os.system('python -m spacy download en_core_web_sm')
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

def text_analysis(text):
    doc = nlp(text)
    html = displacy.render(doc, style="dep", page=True)
    html = (
        "<div style='max-width:100%; max-height:360px; overflow:auto'>"
        + html
        + "</div>"
    )
    pos_count = {
        "char_count": len(text),
        "token_count": 0,
    }
    pos_tokens = []

    for token in doc:
        pos_tokens.extend([(token.text, token.pos_), (" ", None)])

    return pos_tokens, pos_count, html

demo = gr.Interface(
    text_analysis,
    gr.Textbox(placeholder="Enter sentence here..."),
    ["highlight", "json", "html"],
    examples=[
        ["What a beautiful morning for a walk!"],
        ["It was the best of times, it was the worst of times."],
    ],
)

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 9.8 MB/s eta 0:00:00:00:010:01m
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.
